In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys
os.environ['PYTHONPATH'] = os.environ['PYTHONPATH'] + 'C:\\Users\\rdpuser\\Documents\\Github\\parkit;'
sys.path.append('C:\\Users\\rdpuser\\Documents\\Github\\parkit')

In [3]:
import logging
import multiprocessing
import queue

import parkit as pk

logger = logging.getLogger(__name__)

pk.import_site('C:\\Users\\rdpuser\\Desktop\\test')

In [ ]:
# Queue

In [ ]:
q1 = queue.Queue()
q2 = pk.Queue('memory')
q3 = multiprocessing.Queue()
q4 = pk.Queue()

In [ ]:
%%timeit
for i in range(1000):
    q1.put_nowait(i)
for i in range(1000):
    q1.get_nowait()

In [ ]:
%%timeit
with pk.transaction(q2):
    for i in range(1000):
        q2.put(i)
    for i in range(1000):
        q2.get()

In [ ]:
%%timeit
for i in range(1000):
    q2.put(i)
for i in range(1000):
    q2.get()

In [ ]:
%%timeit
for i in range(1000):
    q3.put(i)
for i in range(1000):
    q3.get()

In [ ]:
# Array

In [4]:
a1 = list()
a2 = pk.Array('memory')
a3 = multiprocessing.Array('i', range(10000))

In [ ]:
%%timeit
for i in range(10000):
    a1.append(i)
a1.clear()

In [7]:
%%timeit
with pk.transaction(a2):
    for i in range(10000):
        a2.append(i)
a2.clear()

81.3 ms ± 1.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
for i in range(10000):
    a3[i] = i

In [ ]:
# Dict

In [ ]:
d1 = dict()
d2 = pk.Dict('memory')
d3 = multiprocessing.Manager().dict()

In [ ]:
data = []
for i in range(10000):
    data.append((i, i))

In [ ]:
%%timeit
for i in range(10000):
    d1[i] = i
d1.clear()

In [ ]:
%%timeit
d1.update(data)
d1.clear()

In [ ]:
%%timeit
with pk.transaction(d2):
    for i in range(10000):
        d2[i] = i
    d2.clear()

In [ ]:
%%timeit
d2.update(data)
d2.clear()

In [ ]:
%%timeit
for i in range(10000):
    d2[i] = i
d2.clear()

In [ ]:
%%timeit
for i in range(10000):
    d3[i] = i
d3.clear()

In [ ]:
%%timeit
d3.update(data)
d3.clear()

In [ ]:
# File

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
rows = []
for i in range(1000000):
    rows.append(dict(a = i, b = i))
df = pd.DataFrame(rows)

In [ ]:
%%timeit
df.to_feather('test')

In [ ]:
f = pk.File('memory', mode = 'wb')

In [ ]:
%%timeit
with f:
    df.to_feather(f)

In [ ]:
%%timeit
_ = pd.read_feather('test')

In [ ]:
%%timeit
f.mode = 'rb'
with f:
    assert df.equals(pd.read_feather(f))

In [ ]:
%%timeit
f.mode = 'rb'
with pk.snapshot(f):
    assert isinstance(f.content, memoryview)

In [ ]:
a = np.zeros((1000,1000))

In [ ]:
f = pk.File(mode = 'wb')
with f:
    np.save(f, a)

In [ ]:
%%timeit
f.mode = 'rb'
with pk.snapshot(f):
    assert isinstance(f.content, memoryview)